In [1]:
# 训练五子棋AI
from __future__ import print_function
import random
import numpy as np
from collections import deque,defaultdict #deque是一个双端队列
from game import Board,Game
from mcts_pure import MCTSPlayer as MCTS_Pure #随机走子策略AI
from mcts_alphaZero import MCTSPlayer #AlphaGo方式AI
from policy_value_net_pytorch import PolicyValueNet #pytorch

In [2]:
class TrainPipeline():
    def __init__(self,init_model = None):
        #设置棋盘和游戏参数
        self.board_width = 10
        self.board_height = 10
        self.n_in_row = 5
        self.board = Board(width = self.board_width,
                          height = self.board_height,
                          n_in_row = self.n_in_row)
        self.game = Game(self.board)
        #设置训练参数
        self.learn_rate = 2e-3 #基准学习率
        self.lr_multiplier = 1.0 #基于KL自动调整学习倍速
        self.temp = 1.0 #温度参数
        self.n_playout = 100 #每下一步棋，模拟的步数
        self.c_puct = 5 #exploitation 和 exploration间的折中系数
        self.buffer_size = 1000
        self.batch_size = 4 #mini-batch size for training
        self.data_buffer = deque(maxlen = self.buffer_size) #使用deque创建一个双端队列
        self.play_batch_size = 1
        self.epochs = 5 # num of train_steps for each update
        self.kl_targ = 0.2 #早停检查
        self.check_freq = 50 #每50次检查一次，策略价值网络是否更新
        self.game_batch_num = 200 #训练多少个epoch
        self.best_win_ratio = 0.0 #判断是否有更好的模型
        #弱AI(纯MCTS)模拟步数，用于给训练的策略AI提供对手
        self.pure_mcts_playout_num = 100
        if init_model:
            #通过init_model设置策略网络
            self.policy_value_net = PolicyValueNet(self.board_width,self.board_height,model_file = init_model)
        else:
            #训练一个新的策略网络
            self.policy_value_net = PolicyValueNet(self.board_width,self.board_height)
        #AI Player,设置is_selfplay = 1,因为是在进行训练
        self.mcts_player = MCTSPlayer(self.policy_value_net.policy_value_fn,
                                     c_puct = self.c_puct,
                                     n_playout = self.n_playout,
                                     is_selfplay = 1)
    #通过旋转和翻转增加数据集，play_data:[(state,mcts_prob,winner_z),...,...]
    def get_equi_data(self,play_data):
        extend_data = []
        for state,mcts_prob,winner in play_data:
            #在四个方向上进行expand,每个方向都进行旋转，水平翻转
            for i in [1,2,3,4]:
                #逆时针旋转
                equi_state = np.array([np.rot90(s,i) for s in state])
                equi_mcts_prob = np.rot90(np.flipud(mcts_prob.reshape(self.board.height,self.board.width)),i)
                extend_data.append((equi_state,np.flipud(equi_mcts_prob).flatten(),winner))
                #水平翻转
                equi_state = np.array([np.fliplr(s) for s in equi_state])
                equi_mcts_prob = np.fliplr(equi_mcts_prob)
                extend_data.append((equi_state,np.flipud(equi_mcts_prob).flatten(),winner))
        return extend_data
    
    #收集自我对弈数据，用于训练
    def collect_selfplay_data(self,n_games =1):
        for i in range(n_games):
            #与MCTSPlayer进行对弈
            winner,play_data = self.game.start_self_play(self.mcts_player,temp = self.temp)
            play_data = list(play_data)[:]
            #保存下了多少步
            self.episode_len = len(play_data)
            #增加数据play_data
            play_data = self.get_equi_data(play_data)
            self.data_buffer.extend(play_data)
            
    #更新策略网络
    def policy_update(self):
        mini_batch = random.sample(self.data_buffer,self.batch_size)
        state_batch = [data[0] for data in mini_batch]
        mcts_probs_batch = [data[1] for data in mini_batch]
        winner_batch = [data[2] for data in mini_batch]
        #保存更新前的old_probs,old_v
        old_probs,old_v = self.policy_value_net.policy_value(state_batch)
        for i in range(self.epochs):
            #每次训练，调整参数，返回loss和entropy
            loss,entropy = self.policy_value_net.train_step(state_batch,
                                                            mcts_probs_batch,
                                                            winner_batch,
                                                            self.learn_rate * self.lr_multiplier)
            #输入状态，得到行动的可能性和状态值，按照batch输入
            new_probs,new_v = self.policy_value_net.policy_value(state_batch)
            #计算更新前后的两次loss差
            kl = np.mean(np.sum(old_probs *(np.log(old_probs + 1e-10) - np.log(new_probs + 1e-10)),axis = 1))
            if kl > self.kl_targ * 4: #early stopping if D_KL diveargs badly
                break
        #动态调整学习倍率lr_multiplier
        if kl > self.kl_targ * 2 and self.lr_multiplier > 0.1:
            self.lr_multiplier /= 1.5
        elif kl < self.kl_targ / 2 and self.lr_multiplier < 10:
            self.lr_multiplier *= 1.5
                
        explained_var_old = (1 - 
                             np.var(np.array(winner_batch) - old_v.flatten()) /
                             np.var(np.array(winner_batch)))
        explained_var_new = (1 -
                            np.var(np.array(winner_batch) - new_v.flatten()) /
                            np.var(np.array(winner_batch)))
        print(("kl:{:.5f},"
              "lr_multiplier:{:.3f},"
              "loss:{},"
               "entropy:{},"
               "explained_var_old:{:.3f},"
              "explained_var_new:{:.3f}"
                  ).format(kl,
                          self.lr_multiplier,
                           loss,
                           entropy,
                           explained_var_old,
                           explained_var_new))
        return loss,entropy
        
    #用于评估训练网络的质量，评估一共10场play,返回比赛胜率(赢1分，输0分，平0.5分)
    def policy_evaluate(self,n_games = 10):
        current_mcts_player = MCTSPlayer(self.policy_value_net.policy_value_fn,
                                        c_puct = self.c_puct,n_playout = self.n_playout)
        pure_mcts_player = MCTS_Pure(c_puct =self.c_puct,n_playout = self.pure_mcts_playout_num)
        win_cnt = defaultdict(int)
        for i in range(n_games):
            #AI和弱AI(纯MCTS)对弈，不需要可视化。is_shown = 0,轮流执黑 start_player = i % 2
            winner = self.game.start_play(current_mcts_player,pure_mcts_player,start_player= i % 2,is_shown = 0)
            win_cnt[winner] += 1
        #计算胜率，平手计为0.5分
        win_ratio = 1.0 *(win_cnt[1] + 0.5 * win_cnt[-1]) / n_games
        print('num_playouts:{},win: {},lose: {},tie: {}'.format(self.pure_mcts_playout_num,
                                                               win_cnt[1],win_cnt[2],win_cnt[-1]))
        return win_ratio
    
    def run(self):
        #开始训练
        try:
            #训练game_batch_num次，每个batch比赛play_batch_size场
            for i in range(self.game_batch_num):
                #收集自我对弈数据
                self.collect_selfplay_data(self.play_batch_size)
                print('batch i:{},episode_len:{}'.format(i+1,self.episode_len))
                if len(self.data_buffer) > self.batch_size:
                    loss,entropy = self.policy_update()
                #判断当前模型表现，保存最优模型
                if (i + 1) % self.check_freq == 0:
                    print('current self-play batch:{}'.format(i+1))
                    win_ratio = self.policy_evaluate()
                    #保存当前策略
                    self.policy_value_net.save_model('./current_policy_model')
                    if win_ratio > self.best_win_ratio:
                        print('发现新的最优策略，进行策略更新')
                        self.best_win_ratio = win_ratio
                        #更新最优策略
                        self.policy_value_net.save_model('./best_policy_model')
                        if (self.best_win_ratio == 1.0 and
                           self.pure_mcts_playout_num < 500):
                            self.pure_mcts_playout_num += 100
                            self.best_win_ratio = 0.0
        except KeyboardInterrupt:
            print('\n\quit')    

In [3]:
if __name__ == '__main__':
    training_pipeline = TrainPipeline()
    training_pipeline.run()    

C:\Users\Administrator\houchangligong\项目简历\L15-2week15五子棋&二手车价格预测\policy_value_net_pytorch.py:56: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  x_act = F.log_softmax(self.act_fc1(x_act))
C:\Users\Administrator\Anaconda3\lib\site-packages\torch\nn\functional.py:1614: UserWarning: nn.functional.tanh is deprecated. Use torch.tanh instead.
  warnings.warn("nn.functional.tanh is deprecated. Use torch.tanh instead.")


batch i:1,episode_len:39
kl:0.00030,lr_multiplier:1.500,loss:5.581817626953125,entropy:4.604724884033203,explained_var_old:0.000,explained_var_new:0.000
batch i:2,episode_len:11
kl:0.00014,lr_multiplier:2.250,loss:5.603337287902832,entropy:4.6046905517578125,explained_var_old:-0.000,explained_var_new:0.000
batch i:3,episode_len:9
kl:0.01294,lr_multiplier:3.375,loss:5.642848491668701,entropy:4.589370250701904,explained_var_old:0.000,explained_var_new:0.000
batch i:4,episode_len:12
kl:1.46082,lr_multiplier:2.250,loss:5.0024237632751465,entropy:4.402594566345215,explained_var_old:0.001,explained_var_new:0.040
batch i:5,episode_len:10
kl:1.26546,lr_multiplier:1.500,loss:7.819894313812256,entropy:4.299123764038086,explained_var_old:-inf,explained_var_new:-inf


C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: RuntimeWarning: divide by zero encountered in double_scalars
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: divide by zero encountered in double_scalars


batch i:6,episode_len:15
kl:0.03592,lr_multiplier:2.250,loss:5.570807456970215,entropy:4.6027727127075195,explained_var_old:-0.013,explained_var_new:0.000
batch i:7,episode_len:11
kl:0.03674,lr_multiplier:3.375,loss:5.355557918548584,entropy:4.582782745361328,explained_var_old:0.000,explained_var_new:0.000
batch i:8,episode_len:9
kl:0.03925,lr_multiplier:5.062,loss:5.60396671295166,entropy:4.569382190704346,explained_var_old:0.000,explained_var_new:0.000
batch i:9,episode_len:11
kl:0.02759,lr_multiplier:7.594,loss:5.47324275970459,entropy:4.565533638000488,explained_var_old:0.000,explained_var_new:0.000
batch i:10,episode_len:9
kl:0.10680,lr_multiplier:7.594,loss:5.42147159576416,entropy:4.477851390838623,explained_var_old:0.000,explained_var_new:0.000
batch i:11,episode_len:9
kl:0.30622,lr_multiplier:7.594,loss:5.659366607666016,entropy:4.6013712882995605,explained_var_old:0.000,explained_var_new:0.000
batch i:12,episode_len:9
kl:0.00041,lr_multiplier:11.391,loss:5.598829746246338,ent

C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:95: RuntimeWarning: invalid value encountered in double_scalars
C:\Users\Administrator\Anaconda3\lib\site-packages\ipykernel_launcher.py:98: RuntimeWarning: invalid value encountered in double_scalars


batch i:38,episode_len:10
kl:0.00210,lr_multiplier:11.391,loss:5.353384017944336,entropy:4.4633355140686035,explained_var_old:0.000,explained_var_new:0.000
batch i:39,episode_len:9
kl:0.00162,lr_multiplier:11.391,loss:5.534691333770752,entropy:4.44608736038208,explained_var_old:0.000,explained_var_new:0.000
batch i:40,episode_len:12
kl:0.00132,lr_multiplier:11.391,loss:5.391848564147949,entropy:4.433286666870117,explained_var_old:0.000,explained_var_new:0.000
batch i:41,episode_len:15
kl:0.00109,lr_multiplier:11.391,loss:4.927127361297607,entropy:4.42595100402832,explained_var_old:nan,explained_var_new:nan
batch i:42,episode_len:13
kl:0.00076,lr_multiplier:11.391,loss:5.6244707107543945,entropy:4.421617031097412,explained_var_old:0.000,explained_var_new:0.000
batch i:43,episode_len:9
kl:0.00080,lr_multiplier:11.391,loss:5.566808700561523,entropy:4.417418956756592,explained_var_old:0.000,explained_var_new:0.000
batch i:44,episode_len:9
kl:0.00123,lr_multiplier:11.391,loss:5.172810077667

batch i:91,episode_len:11
kl:0.00167,lr_multiplier:11.391,loss:4.30312442779541,entropy:3.424791097640991,explained_var_old:0.000,explained_var_new:0.000
batch i:92,episode_len:9
kl:0.00154,lr_multiplier:11.391,loss:4.423163414001465,entropy:3.4242382049560547,explained_var_old:nan,explained_var_new:nan
batch i:93,episode_len:11
kl:0.00119,lr_multiplier:11.391,loss:5.0746636390686035,entropy:3.4398577213287354,explained_var_old:0.000,explained_var_new:0.000
batch i:94,episode_len:9
kl:0.00259,lr_multiplier:11.391,loss:4.1134562492370605,entropy:3.455568790435791,explained_var_old:0.000,explained_var_new:0.000
batch i:95,episode_len:9
kl:0.00247,lr_multiplier:11.391,loss:4.94969367980957,entropy:3.465022325515747,explained_var_old:0.000,explained_var_new:0.000
batch i:96,episode_len:9
kl:0.00299,lr_multiplier:11.391,loss:4.186903476715088,entropy:3.478334426879883,explained_var_old:0.000,explained_var_new:0.000
batch i:97,episode_len:9
kl:0.00281,lr_multiplier:11.391,loss:4.435372352600

batch i:144,episode_len:9
kl:0.00125,lr_multiplier:11.391,loss:4.204497337341309,entropy:3.2653861045837402,explained_var_old:0.000,explained_var_new:0.000
batch i:145,episode_len:9
kl:0.00149,lr_multiplier:11.391,loss:4.03969144821167,entropy:3.2536633014678955,explained_var_old:0.000,explained_var_new:0.000
batch i:146,episode_len:9
kl:0.00068,lr_multiplier:11.391,loss:4.113523483276367,entropy:3.2480673789978027,explained_var_old:0.000,explained_var_new:0.000
batch i:147,episode_len:9
kl:0.00026,lr_multiplier:11.391,loss:4.011798858642578,entropy:3.246997356414795,explained_var_old:nan,explained_var_new:nan
batch i:148,episode_len:9
kl:0.00096,lr_multiplier:11.391,loss:4.111180305480957,entropy:3.2496848106384277,explained_var_old:0.000,explained_var_new:0.000
batch i:149,episode_len:9
kl:0.00139,lr_multiplier:11.391,loss:4.592319965362549,entropy:3.2623963356018066,explained_var_old:0.000,explained_var_new:0.000
batch i:150,episode_len:9
kl:0.00097,lr_multiplier:11.391,loss:4.42524

batch i:197,episode_len:10
kl:0.00164,lr_multiplier:11.391,loss:4.106949329376221,entropy:3.232184886932373,explained_var_old:0.000,explained_var_new:0.000
batch i:198,episode_len:9
kl:0.00083,lr_multiplier:11.391,loss:3.940821886062622,entropy:3.2221779823303223,explained_var_old:0.000,explained_var_new:0.000
batch i:199,episode_len:9
kl:0.00062,lr_multiplier:11.391,loss:4.351642608642578,entropy:3.2169551849365234,explained_var_old:0.000,explained_var_new:0.000
batch i:200,episode_len:9
kl:0.00056,lr_multiplier:11.391,loss:3.8383612632751465,entropy:3.216578245162964,explained_var_old:0.000,explained_var_new:0.000
current self-play batch:200
num_playouts:100,win: 1,lose: 9,tie: 0
